In [1]:
import numpy as np

In [2]:
# Read the data and remove the first Row
train_data = np.genfromtxt('train_features.csv', delimiter = ',') 
train_data = train_data[1:]
test_data = np.genfromtxt('test_features.csv', delimiter = ',') 
test_data = test_data[1:]

In [3]:
# Make an array of data arrays X = [[x00,x01,...,x09],...]
X_train = np.array([row[2:] for row in train_data])
X_test = np.array([row[2:] for row in test_data])

In [4]:
# Make group of 12 hour vectors
X_subgroups = np.array_split(X_train, len(X_train)/12)
X_subgroups_test = np.array_split(X_test, len(X_test)/12)

In [5]:
def add_together(list1, list2):
    for i, (a, b) in enumerate(zip(list1, list2)):
        list1[i] = a+b
        
def divide(x,y):
    if y != 0:
        return x / y 
    else: 
        return 0

def preprocessing(X_subgroups):
    X_avg = []
    X_total = [0] * 35
    X_nums = [0] * 35
    for sub in X_subgroups:
        X_curr = [0] * 35
        X_curr_nums = [0] * 35
        for sublist in sub:
            i = -1
            for item in sublist:
                i += 1
                if item == item:
                    X_curr[i] += item
                    X_curr_nums[i] += 1
        X_avg.append(['nan' if X_curr_nums[i] == 0 else X_curr[i]/X_curr_nums[i] for i in range(34)])

        add_together(X_total, X_curr)
        add_together(X_nums, X_curr_nums)

    X_avg_total = list(map(lambda x, y: divide(x, y), X_total, X_nums))

    for i, person in enumerate(X_avg):
        for j, value in enumerate(person):
            if value == 'nan':
                X_avg[i][j] = X_avg_total[j]
    return X_avg

In [6]:
"""for i, sub in enumerate(X_subgroups):
    for j, sublist in enumerate(sub):
        for k, item in enumerate(sublist):
            if item != item:
                X_subgroups[i][j][k] = X_avg_total[k]"""

'for i, sub in enumerate(X_subgroups):\n    for j, sublist in enumerate(sub):\n        for k, item in enumerate(sublist):\n            if item != item:\n                X_subgroups[i][j][k] = X_avg_total[k]'

In [7]:
X_avg_train = preprocessing(X_subgroups)

In [15]:
X_avg_test = preprocessing(X_subgroups_test)

In [9]:
# Read the data and remove the first Row
train_data = np.genfromtxt('train_labels.csv', delimiter = ',') 
train_data = train_data[1:]

In [10]:
Y = np.array([row[1:12] for row in train_data])
Y = Y.transpose()

In [11]:
from sklearn.linear_model import LinearRegression
models = []
for y in Y:
    models.append(LinearRegression().fit(X_avg_train, y))

In [21]:
X_avg_test

[[39.0,
  32.560431985294116,
  41.35,
  18.5,
  2.7431940626717988,
  36.0,
  9.649999999999999,
  14.5,
  -6.5,
  11.714285714285714,
  147.0,
  5.4,
  18.0,
  2.3,
  28.0,
  31.5,
  0.53,
  193.5,
  92.98143021914649,
  217.5,
  80.75,
  2.35,
  4.15,
  64.5,
  8.15,
  114.0,
  100.0,
  1.460460251046026,
  100.0,
  28.45,
  84.5,
  24.1,
  7.41073714054276,
  128.625],
 [62.0,
  32.560431985294116,
  39.93407120968931,
  23.438466686496135,
  2.7431940626717988,
  37.25,
  10.655362426035511,
  23.494027546900977,
  -1.1293813264843633,
  16.666666666666668,
  263.74157099697885,
  3.6120153525891205,
  11.638303928487417,
  1.4785751800084697,
  41.122433542251976,
  251.06665813715455,
  0.5533041182088841,
  207.103659563716,
  92.98143021914649,
  142.7046269953656,
  91.55555555555556,
  2.012245708970998,
  4.15366500131476,
  82.05555555555556,
  7.158563740329674,
  99.0915565915566,
  94.2,
  1.460460251046026,
  106.32170542635659,
  31.373145377829097,
  111.4,
  1.67071

In [35]:
results = []
for person_data in X_avg_test[0:1]:
    predictions = []
    person_data = np.array(person_data)
    print(person_data)
    for model in models:
         predictions.append(model.predict(person_data))
    results.append(predictions)

[ 39.          32.56043199  41.35        18.5          2.74319406
  36.           9.65        14.5         -6.5         11.71428571
 147.           5.4         18.           2.3         28.
  31.5          0.53       193.5         92.98143022 217.5
  80.75         2.35         4.15        64.5          8.15
 114.         100.           1.46046025 100.          28.45
  84.5         24.1          7.41073714 128.625     ]


ValueError: Expected 2D array, got 1D array instead:
array=[ 39.          32.56043199  41.35        18.5          2.74319406
  36.           9.65        14.5         -6.5         11.71428571
 147.           5.4         18.           2.3         28.
  31.5          0.53       193.5         92.98143022 217.5
  80.75         2.35         4.15        64.5          8.15
 114.         100.           1.46046025 100.          28.45
  84.5         24.1          7.41073714 128.625     ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [27]:
results

[[array([0.42893847]),
  array([0.16093284]),
  array([0.18746615]),
  array([0.19035445]),
  array([0.19355782]),
  array([0.24199958]),
  array([-0.03645167]),
  array([0.28315898]),
  array([0.04525855]),
  array([0.04764246]),
  array([0.05808356])],
 [array([0.10325795]),
  array([0.00921827]),
  array([0.19461913]),
  array([0.18684186]),
  array([0.18658941]),
  array([0.08169393]),
  array([0.17820423]),
  array([0.08161623]),
  array([0.01220489]),
  array([0.05928968]),
  array([0.02955841])],
 [array([0.34106821]),
  array([0.07874091]),
  array([0.28525921]),
  array([0.27711227]),
  array([0.27503966]),
  array([0.20510433]),
  array([0.10664288]),
  array([0.21153092]),
  array([0.0391961]),
  array([0.03009695]),
  array([0.05310238])],
 [array([0.09490746]),
  array([0.09891136]),
  array([0.1603142]),
  array([0.16532668]),
  array([0.17313437]),
  array([0.31034321]),
  array([0.08993941]),
  array([0.3130643]),
  array([0.03061854]),
  array([0.24514442]),
  array([0